In [1]:
!pip install PyDrive
!pip install rouge

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import json, gzip
import torch
import torch.nn as nn
import numpy as np
import time
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import pickle
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import re
import unicodedata

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

##Mixed data

In [0]:
# # Data for training set = 10k dev set = 1k
# download = drive.CreateFile({'id': '1yJB36Jj1jw3U0JwE3IwhBYDY3VI1oCGd'})
# download.GetContentFile('train.p')
# download = drive.CreateFile({'id': '1T6YxqByvjNlCSZrU7FVmdCYlxM0RHlgw'})
# download.GetContentFile('dev.p')
# download = drive.CreateFile({'id': '1vPOGkgumTOVQ4N84GBo8AAPeAgwfyMep'})
# download.GetContentFile('index2word.p')
# download = drive.CreateFile({'id': '1JRYv-RQqhX6-fdYVUmyZmP9tBOtZjp_w'})
# download.GetContentFile('word2index.p')

In [0]:
#Data for training set = 50k dev set = 5k
# download = drive.CreateFile({'id': '1xq4JlZoWZ8_eO5F6VNBQvk2Mvub1Etpb'})
# download.GetContentFile('trainL.p')
# download = drive.CreateFile({'id': '1G3YAtOtrfv3PswNSHyGm0SIZy60n95zU'})
# download.GetContentFile('devL.p')
# download = drive.CreateFile({'id': '1bZXADxMyEn-yWtneA1sxDouGpuce1LRh'})
# download.GetContentFile('index2wordL.p')
# download = drive.CreateFile({'id': '1IMnW7Oyq4q9Ho5z87MbmVwlODjzEvY9h'})
# download.GetContentFile('word2indexL.p')


In [0]:
# #Data with Extractive/Abstractive lable
download = drive.CreateFile({'id': '1LOzhGMuW8F4ChPUsycBjEOwpLDQPeD0I'})
download.GetContentFile('trainNew.p')
download = drive.CreateFile({'id': '1yoaCyqZgZ5QszX0g4ABnNV-9p-D6DUlZ'})
download.GetContentFile('devNew.p')
download = drive.CreateFile({'id': '1XSaEgXAwPWTojc6ZjzIy8vR38jjiCUSW'})
download.GetContentFile('testNew.p')
download = drive.CreateFile({'id': '1U45R7EOqcJCUQAyHMMRmhFgWoxfzxyL8'})
download.GetContentFile('index2wordNew.p')
download = drive.CreateFile({'id': '1DGnwq99nJAq9i9TQ45HPjaO3aYWqDDEL'})
download.GetContentFile('word2indexNew.p')

In [0]:
# train_dict = pickle.load( open( "train.p", "rb" ) )
# dev_dict = pickle.load( open( "dev.p", "rb" ) )
# index2word = pickle.load( open( "index2word.p", "rb" ) )
# word2index = pickle.load( open( "word2index.p", "rb" ) )

In [0]:
train_dict = pickle.load( open( "trainNew.p", "rb" ) )
dev_dict = pickle.load( open( "devNew.p", "rb" ) )
test_dict = pickle.load( open( "testNew.p", "rb" ) )
index2word = pickle.load( open( "index2wordNew.p", "rb" ) )
word2index = pickle.load( open( "word2indexNew.p", "rb" ) )

In [8]:
# data_code = "extractive"
# data_code = "abstractive"
data_code = "mixed"
filtered_train_dict = []
filtered_dev_dict = []
filtered_test_dict = []

for ind in range(len(train_dict)):
    data = train_dict[ind]
    if data['density_bin'] == data_code:
        filtered_train_dict.append(data)
for ind in range(len(dev_dict)):
    data = dev_dict[ind]
    if data['density_bin'] == data_code:
        filtered_dev_dict.append(data)
for ind in range(len(test_dict)):
    data = test_dict[ind]
    if data['density_bin'] == data_code:
        filtered_test_dict.append(data)

print(len(filtered_train_dict))
print(len(filtered_dev_dict))
print(len(filtered_test_dict))

3554
374
349


In [0]:
#Extractive summary for 100 data points
# download = drive.CreateFile({'id': '1N6K91ySpywAB9pSKVCoJZKBn84S-2UFx'})
# download.GetContentFile('extract_train100.p')
# train_dict = pickle.load( open( "extract_train100.p", "rb" ) )

In [0]:
train_dict = filtered_train_dict[:]
dev_dict = filtered_dev_dict[:]
test_dict = filtered_test_dict[:]

In [10]:
train_dict_text = []
for i, data in enumerate(train_dict):
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
#         if i==789:
#             print(s, summ)
    t['summary'] = summ
    train_dict_text.append(t)
print(len(train_dict_text))

dev_dict_text = []
for data in dev_dict:
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
    t['summary'] = summ
    dev_dict_text.append(t)
print(len(dev_dict_text))

3554
374


In [11]:
ind = 0
print(train_dict[ind]['summary'])
print(train_dict_text[ind]['summary'])
print(index2word[train_dict[ind]['summary'][-2]])

[19835, 19, 38, 112, 75154, 6405, 108, 18, 109, 110, 242, 10, 10, 10, 42803, 50]
advertise in the daily telegraph; product information and key facts for    advertisers .
advertisers


In [12]:
import nltk
import nltk.data
from nltk import tokenize
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pickle
import json, gzip
import time
import math
from sklearn.cluster import KMeans
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import euclidean
nltk.download('punkt')

W0508 22:21:00.165590 140182069344128 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [14]:
embed = hub.Module(module_url)
tf.logging.set_verbosity(tf.logging.ERROR)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Instructions for updating:
Colocations handled automatically by placer.


W0508 22:21:05.634465 140182069344128 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    embed = hub.Module(module_url)
    embedded_text = embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

session = tf.Session(graph=g)
session.run(init_op)

In [16]:
ext_train_data = []
ext_dev_data = []

for i, data in enumerate(train_dict_text):
    if i%1000==0:
        print(i)
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    try:
        sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
        num_sent = sentences_embeddings.shape[0]
        k = round(math.sqrt(num_sent))
        clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
        cluster_centers = clustering.cluster_centers_
        center_dist = defaultdict(list)
        for ns in range(num_sent):
            label = clustering.labels_[ns]
            dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
            center_dist[label].append((dist, ns))

        final = []
        for labels in range(k):
            try:
                center_dist[labels].sort(key = lambda x: x[0])
                final.append(center_dist[labels][0][1])
            except:
                continue
        final.sort()
        ex = []
        for i in final:
            ex.append(sentences_list[i])
        extractive = (' ').join(ex)
        dict_ = {}
        dict_['text'] = extractive[:]
        dict_['summary'] = data['summary'][:]
        ext_train_data.append(dict_)
    except:
        print('error')
end = time.time()
print(end-start)

for i, data in enumerate(dev_dict_text):
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    
    sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
    num_sent = sentences_embeddings.shape[0]
    k = round(math.sqrt(num_sent))
    clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
    cluster_centers = clustering.cluster_centers_
    center_dist = defaultdict(list)
    for ns in range(num_sent):
        label = clustering.labels_[ns]
        dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
        center_dist[label].append((dist, ns))
            
    final = []
    for labels in range(k):
        try:
            center_dist[labels].sort(key = lambda x: x[0])
            final.append(center_dist[labels][0][1])
        except:
            continue
    final.sort()
    ex = []
    for i in final:
        ex.append(sentences_list[i])
    extractive = (' ').join(ex)
    dict_ = {}
    dict_['text'] = extractive[:]
    dict_['summary'] = data['summary'][:]
    ext_dev_data.append(dict_)
end = time.time()
print(end-start)

0
1000
2000
3000
0.030985593795776367
0.03412151336669922


In [0]:
ext_train_data = sorted(ext_train_data, key = lambda i: len(i['text']), reverse = True)
# dev_dict = sorted(dev_dict, key = lambda i: len(i['text']), reverse = True)

In [17]:
ind = 1
print(ext_train_data[ind]['text'])
print(ext_train_data[ind]['summary'])
# print(ext_dev_data[ind]['text'])
# print(ext_dev_data[ind]['summary'])

vincent 's hospital saturday afternoon after a female caller placed a call to 911 . following alexa ray 's hospitalization , her rep issued a statement saying , "she is stable and her doctors are assessing her needs ."
the singer releases a statement about his daughter alexa ray 's hospitalization


In [0]:
from rouge import Rouge

def calculate_rouge (hyps, refs):
    #print(hyps, refs)
    r = Rouge()
    scores = r.get_scores(hyps, refs, avg=True)
    return(scores)


In [0]:
# clustered = 'Colab is a good platform. It is very useful.'
# org = 'Colab is a very useful platform.'
# # org = 'Colab is a good platform. It is very useful.'
# scores = calculate_rouge (org, clustered)
# print(scores)
# scores = calculate_rouge (clustered, org)
# print(scores)

In [19]:
ext_cluster_summ = []
ext_ref_summ = []
for data in ext_train_data:
    ext_cluster_summ.append(data['text'][:])
    ext_ref_summ.append(data['summary'][:])
print(len(ext_cluster_summ))
print(len(ext_ref_summ))

3554
3554


In [20]:
cluster_summ = []
ref_summ = []
for data in train_dict_text:
    if data['text'] != '.'*len(data['text']) and data['text']:
        cluster_summ.append(data['text'][:])
        ref_summ.append(data['summary'][:])
print(len(cluster_summ))
print(len(ref_summ))

3554
3554


In [21]:
scores = calculate_rouge (ext_cluster_summ, ext_ref_summ)
print(scores)
scores = calculate_rouge (cluster_summ, ref_summ)
print(scores)

{'rouge-1': {'f': 0.2471733932078895, 'p': 0.17181656777216944, 'r': 0.4972278861199391}, 'rouge-2': {'f': 0.1058415587228665, 'p': 0.07158652511643064, 'r': 0.23941145137410924}, 'rouge-l': {'f': 0.1637220972708498, 'p': 0.15212461188447737, 'r': 0.44256714155898563}}
{'rouge-1': {'f': 0.21372285066313368, 'p': 0.1261209219823455, 'r': 0.8062798294253963}, 'rouge-2': {'f': 0.09858325721230965, 'p': 0.056006456293525346, 'r': 0.49933411222768703}, 'rouge-l': {'f': 0.12089856620104618, 'p': 0.11747929034473187, 'r': 0.7565068407820615}}
